## Подготовка 

### Определяем переменные окружения

In [1]:
# KAFKA
%env KAFKA_BROKER = rc1b-2erh7b35n4j4v869.mdb.yandexcloud.net:9091
%env KAFKA_TOPIC_IN = student.topic.cohort29.8ubble8uddy_in03
%env KAFKA_TOPIC_OUT = student.topic.cohort29.8ubble8uddy_out03
%env KAFKA_USER = de-student
%env KAFKA_PASSWORD = ltcneltyn
# POSTGRESQL SOURCE
%env PG_SOURCE_URL = rc1a-fswjkpli01zafgjm.mdb.yandexcloud.net:6432/de
%env PG_SOURCE_USER = student
%env PG_SOURCE_PASSWORD = de-student
%env PG_SOURCE_TABLE = public.subscribers_restaurants
# POSTGRESQL TARGET
%env PG_TARGET_URL = localhost:5432/de
%env PG_TARGET_USER = jovyan
%env PG_TARGET_PASSWORD = jovyan
%env PG_TARGET_TABLE = public.subscribers_feedback
# SPARK
%env PYSPARK_PYTHON = /usr/bin/python3

env: KAFKA_BROKER=rc1b-2erh7b35n4j4v869.mdb.yandexcloud.net:9091
env: KAFKA_TOPIC_IN=student.topic.cohort29.8ubble8uddy_in03
env: KAFKA_TOPIC_OUT=student.topic.cohort29.8ubble8uddy_out03
env: KAFKA_USER=de-student
env: KAFKA_PASSWORD=ltcneltyn
env: PG_SOURCE_URL=rc1a-fswjkpli01zafgjm.mdb.yandexcloud.net:6432/de
env: PG_SOURCE_USER=student
env: PG_SOURCE_PASSWORD=de-student
env: PG_SOURCE_TABLE=public.subscribers_restaurants
env: PG_TARGET_URL=localhost:5432/de
env: PG_TARGET_USER=jovyan
env: PG_TARGET_PASSWORD=jovyan
env: PG_TARGET_TABLE=public.subscribers_feedback
env: PYSPARK_PYTHON=/usr/bin/python3


### Создаём таблицу-приёмник в PostgreSQL

In [2]:
%%bash
psql postgresql://$PG_TARGET_USER:$PG_TARGET_PASSWORD@$PG_TARGET_URL \
    -c "CREATE TABLE $PG_TARGET_TABLE ( \
        id serial4 NOT NULL, \
        restaurant_id text NOT NULL, \
        adv_campaign_id text NOT NULL, \
        adv_campaign_content text NOT NULL, \
        adv_campaign_owner text NOT NULL, \
        adv_campaign_owner_contact text NOT NULL, \
        adv_campaign_datetime_start int8 NOT NULL, \
        adv_campaign_datetime_end int8 NOT NULL, \
        datetime_created int8 NOT NULL, \
        client_id text NOT NULL, \
        trigger_datetime_created int4 NOT NULL, \
        feedback varchar NULL, \
        CONSTRAINT id_pk PRIMARY KEY (id));"

CREATE TABLE


### Создаём сообщение для Kafka

In [3]:
%%python3
from datetime import datetime, timedelta

current_timestamp = int(round(datetime.now().timestamp()))
key = "first_message"
value = {
    "restaurant_id": "123e4567-e89b-12d3-a456-426614174000",
    "adv_campaign_id": "123e4567-e89b-12d3-a456-426614174003",
    "adv_campaign_content": "first_campaign",
    "adv_campaign_owner": "Ivanov Ivan Ivanovich",
    "adv_campaign_owner_contact": "iiivanov@restaurant.ru",
    "adv_campaign_datetime_start": current_timestamp,
    "adv_campaign_datetime_end": current_timestamp + timedelta(hours=3).seconds,
    "datetime_created": current_timestamp
}

with open('message.txt', 'w') as file:
    file.write(f'{key}: {value}')

### Отправляем сообщение во входной топик Kafka с задержкой в 5 минут

In [4]:
%%bash --bg
sleep "5m"
kafkacat -b $KAFKA_BROKER -t $KAFKA_TOPIC_IN -P -T -K: -l message.txt \
    -X security.protocol=SASL_SSL \
    -X sasl.mechanisms=SCRAM-SHA-512 \
    -X sasl.username="$KAFKA_USER" \
    -X sasl.password="$KAFKA_PASSWORD" \
    -X ssl.ca.location=/usr/local/share/ca-certificates/Yandex/YandexCA.crt

## Действие

### Запускаем Spark-приложение для потоковой обработки данных

In [5]:
%%bash
spark-submit --packages "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0,org.postgresql:postgresql:42.4.0" app.py

24/12/24 21:16:13 WARN Utils: Your hostname, fhm5g0hjoec9p2prfs80 resolves to a loopback address: 127.0.1.1; using 10.128.0.23 instead (on interface eth0)
24/12/24 21:16:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b1557a0d-573f-4ebf-91cd-78d985264afb;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.0 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commo

24/12/24 21:16:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/24 21:17:07 INFO SparkContext: Running Spark version 3.3.0
24/12/24 21:17:08 INFO ResourceUtils: ==============================================================
24/12/24 21:17:08 INFO ResourceUtils: No custom resources configured for spark.driver.
24/12/24 21:17:08 INFO ResourceUtils: ==============================================================
24/12/24 21:17:08 INFO SparkContext: Submitted application: join stream
24/12/24 21:17:08 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amount: 1.0)
24/12/24 21:17:08 INFO ResourceProfile: Limiting resource is cpu
24/12/24 21:17:08 INFO ResourceProfileManager: 

24/12/24 21:21:45 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-39687d75-dd79-4eec-889f-9c98ac6b4a91--283479167-driver-0-1, groupId=spark-kafka-source-39687d75-dd79-4eec-889f-9c98ac6b4a91--283479167-driver-0] Seeking to LATEST offset of partition student.topic.cohort29.8ubble8uddy_in03-0
24/12/24 21:21:45 INFO SubscriptionState: [Consumer clientId=consumer-spark-kafka-source-39687d75-dd79-4eec-889f-9c98ac6b4a91--283479167-driver-0-1, groupId=spark-kafka-source-39687d75-dd79-4eec-889f-9c98ac6b4a91--283479167-driver-0] Resetting offset for partition student.topic.cohort29.8ubble8uddy_in03-0 to position FetchPosition{offset=1, offsetEpoch=Optional.empty, currentLeader=LeaderAndEpoch{leader=Optional[rc1b-d887g1d44vnv8igt.mdb.yandexcloud.net:9091 (id: 9 rack: ru-central1-b)], epoch=0}}.
24/12/24 21:21:45 INFO MicroBatchExecution: Streaming query made progress: {
  "id" : "3ac65a53-d0bd-40de-8d5f-aadd01b931ad",
  "runId" : "5d478219-03f4-424d-b2d3-16413c4ebaf4",
  "n

## Проверка

### Читаем выходной топик в Kafka

In [6]:
%%bash
kafkacat -b $KAFKA_BROKER -t $KAFKA_TOPIC_OUT -C -e -o beginning \
    -X security.protocol=SASL_SSL \
    -X sasl.mechanisms=SCRAM-SHA-512 \
    -X sasl.username="$KAFKA_USER" \
    -X sasl.password="$KAFKA_PASSWORD" \
    -X ssl.ca.location=/usr/local/share/ca-certificates/Yandex/YandexCA.crt

% Reached end of topic student.topic.cohort29.8ubble8uddy_out03 [0] at offset 9: exiting


{"restaurant_id":"123e4567-e89b-12d3-a456-426614174000","adv_campaign_id":"123e4567-e89b-12d3-a456-426614174003","adv_campaign_content":"first_campaign","adv_campaign_owner":"Ivanov Ivan Ivanovich","adv_campaign_owner_contact":"iiivanov@restaurant.ru","adv_campaign_datetime_start":1735074881,"adv_campaign_datetime_end":1735085681,"datetime_created":1735074881,"client_id":"323e4567-e89b-12d3-a456-426614174000","trigger_datetime_created":1735075191}
{"restaurant_id":"123e4567-e89b-12d3-a456-426614174000","adv_campaign_id":"123e4567-e89b-12d3-a456-426614174003","adv_campaign_content":"first_campaign","adv_campaign_owner":"Ivanov Ivan Ivanovich","adv_campaign_owner_contact":"iiivanov@restaurant.ru","adv_campaign_datetime_start":1735074881,"adv_campaign_datetime_end":1735085681,"datetime_created":1735074881,"client_id":"723e4567-e89b-12d3-a456-426614174000","trigger_datetime_created":1735075191}
{"restaurant_id":"123e4567-e89b-12d3-a456-426614174000","adv_campaign_id":"123e4567-e89b-12d3-a4

### Выводим содержимое таблицы-приёмника PostgreSQL

In [7]:
%%bash
psql postgresql://$PG_TARGET_USER:$PG_TARGET_PASSWORD@$PG_TARGET_URL -c "SELECT * FROM $PG_TARGET_TABLE;"

 id |            restaurant_id             |           adv_campaign_id            | adv_campaign_content |  adv_campaign_owner   | adv_campaign_owner_contact | adv_campaign_datetime_start | adv_campaign_datetime_end | datetime_created |              client_id               | trigger_datetime_created | feedback 
----+--------------------------------------+--------------------------------------+----------------------+-----------------------+----------------------------+-----------------------------+---------------------------+------------------+--------------------------------------+--------------------------+----------
  1 | 123e4567-e89b-12d3-a456-426614174000 | 123e4567-e89b-12d3-a456-426614174003 | first_campaign       | Ivanov Ivan Ivanovich | iiivanov@restaurant.ru     |                  1735074881 |                1735085681 |       1735074881 | 323e4567-e89b-12d3-a456-426614174000 |               1735075191 | 
  2 | 123e4567-e89b-12d3-a456-426614174000 | 123e4567-e89b-12d3-a456-4